In [ ]:
import pandas as pd
import yfinance as yf
from datetime import timedelta
import matplotlib.pyplot as plt

def load_file(file_path):
    try:
        df = pd.read_csv(file_path, sep=';')
        return df

    except Exception as e:
        print(f"Error in load_file: {e}")
        return pd.DataFrame()

def merge_df(file_1, file_2): #sprawdzić wyjątki
    try:
        df = pd.concat([file_1, file_2], ignore_index=True)
        return df
    except Exception as e:
        print(f"Error in merge_df: {e}")
        return pd.DataFrame()

def format_symbol_for_yf(symbol): #wrzucić do kontrolera
    if symbol.endswith('.UK'):
        return symbol[:-3] + '.L'
    elif symbol.endswith('.US'):
        return symbol[:-3]
    elif symbol.endswith('.FR'):
        return symbol[:-3] + '.PA'
    return symbol

def format_currency_for_yf(symbol): #wrzucić do kontrolera
    if symbol.endswith('.UK'):
        return 'GBP'
    elif symbol.endswith('.US'):
        return 'USD'
    elif symbol.endswith('.FR' or '.DE'):
        return 'EUR'
    return symbol

def format_df(df): #sprawdzić wszystkie przypadki złego formatu, zrobić wyjątki
    try:
        df['Formatted Symbol'] = df['Symbol'].apply(format_symbol_for_yf)
        df['Currency'] = df['Symbol'].apply(format_currency_for_yf)
        df['Open time'] = pd.to_datetime(df['Open time'], dayfirst=True).dt.normalize()
        
        df['Close time'] = pd.to_datetime(df['Close time'], dayfirst=True, errors='coerce').dt.normalize()
        df['Close time'].fillna(pd.Timestamp.now().normalize(), inplace=True)

    except Exception as e:
        print(f"Error in format_df: {e}")
        return pd.DataFrame()

def load_currencies(df):
    try:
        currencies = []
        for currency in df['Currency'].unique():
            currencies.append(currency)
        return currencies

    except Exception as e:
        print(f"Error in load_currencies: {e}")
        return pd.DataFrame()

def load_exchange_rates(currency, date):
    ticker = f"{currency}PLN=X"
    data = yf.download(ticker, period="1d", start=date, end=date + timedelta(days=1), auto_adjust=False)
    exchange_rate = data["Close"].iloc[-1].item() if not data.empty else None
    return exchange_rate

def stock_timeframe(ticker, df):
    try:
        filtered_df = df[df['Formatted Symbol'] == f'{ticker}']        
        start_date = min(filtered_df['Open time'])
        end_date = max(filtered_df['Close time'])
        business_days = pd.date_range(start=start_date.date(), end=end_date.date(), freq='B') #nie działa dla crypto!
        timeframe = pd.DataFrame({'Date': business_days, 'Volume': 0.0, 'Profit': 0.0, 'Profit cumulated': 0.0, 'Daily change': 0.0, 'Total change': 0.0}) #dodać in PLN (ryzyko walutowe) każdego profitu

        return timeframe
    
    except Exception as e:
        print(f"Error in stock_timeframe: {e}")
        return pd.DataFrame()
    
def load_position(stock_timeframe, position): #obsługa df pozycji, zwracanie do danych o spółce śrendiej ceny zakupu i kursu
    try:
        filtered_timeframe = stock_timeframe[
            (stock_timeframe['Date'] >= position['Open time']) & (stock_timeframe['Date'] <= position['Close time'])]
        for index, row in filtered_timeframe.iterrows():
            try:
                date = row['Date']
                data = yf.download(position['Formatted Symbol'], start=date.strftime('%Y-%m-%d'),
                                   end=(date + timedelta(days=1)).strftime('%Y-%m-%d'), auto_adjust=False)
                stock_price = data['Close'].iloc[0] if isinstance(data['Close'].iloc[0], float) else data['Close'].iloc[
                    0].item()
    
                # exchange_rate = load_exchange_rates(stock['Currency'], date)
                # if isinstance(exchange_rate, pd.Series):
                #     exchange_rate = exchange_rate.iloc[0]
    
                purchase_value = float(position['Open price'].replace(',', '.'))
                sale_value = stock_price
                profit = sale_value - purchase_value
                stock_timeframe.loc[index, 'Profit'] += profit
            
            except Exception as e:
                # print(f"Error retrieving data for {position['Formatted Symbol']} on {date}: {e}")
                # return None
                stock_timeframe.drop(index, inplace=True)  # Usuń wiersz w przypadku błędu
                continue

    except Exception as e:
        print(f"Error in load_position: {e}")
        return pd.DataFrame()
        
def stock_history(ticker,df):
    try:
        timeframe = stock_timeframe(ticker, df)
        positions = stock_positions(ticker, df)

        for _, stock in positions.iterrows():
            timeframe.loc[(timeframe['Date'] >= stock['Open time']) &
                                (timeframe['Date'] <= stock['Close time']), 'Volume'] += 1.0
            load_position(timeframe, stock)
        
        return timeframe
    
    except Exception as e:
        print(f"Error in stock_history: {e}")
    
def stock_positions(ticker, df):
    try:
        positions = df[df['Formatted Symbol'] == ticker]
        return positions
    
    except Exception as e:
        print(f"Error in stock_positions: {e}")
        return pd.DataFrame()
            
def all_stocks_information(unique_tickers):
    try:
        stocks = []
        for ticker in unique_tickers:
            try:
                informations = stock_information(ticker)
                stocks.append({'Ticker': ticker, 'Informations': informations})

            except Exception as e:
                print(f"Error in all_stocks_history in {ticker}: {e}")
                continue
            
        return stocks
    
    except Exception as e:
        print(f"Error in all_stocks_history: {e}")
        return pd.DataFrame()
    
def stock_information(ticker):
    informations = []
    try:
        timeframe = stock_history(ticker, df)
        positions = stock_positions(ticker, df)
        # fast_info = yf.Ticker(ticker).fast_info #uzupełnić potrzebne
        mean_price = ...
        mean_exchange_ratio = ...
        informations.append({'Fast info': ..., 'Timeframe': timeframe, 'Positions': positions, 'Mean price': mean_price, 'Mean exchange ratio': mean_exchange_ratio})
        
        return informations
    
    except Exception as e:
        print(f"Error in stock_information: {e}")
        return pd.DataFrame()
    
def load_summary_timeframe(all_stocks):
    try:
        start_date = min(
            info['Timeframe']['Date'].min() for stock in all_stocks for info in stock['Informations']
        )
        end_date = max(
            info['Timeframe']['Date'].max() for stock in all_stocks for info in stock['Informations']
        )
        business_days = pd.date_range(start=start_date.date(), end=end_date.date())
        timeframe = pd.DataFrame({'Date': business_days, 'Profit': 0.0, 'Profit cumulated': 0.0, 'Daily change': 0.0, 'Total change': 0.0}) #dodać in PLN (ryzyko walutowe) każdego profitu
        
        return timeframe

    except Exception as e:
        print(f"Error in summary_timeframe: {e}")
        return pd.DataFrame()
    
def load_summary(all_stocks):
    summary_timeframe = load_summary_timeframe(all_stocks)
    try:
        for stock in all_stocks:
            for info in stock['Informations']:
                timeframe = info['Timeframe']
                
                for index, row in timeframe.iterrows():
                    date = row['Date']
                    profit = row['Profit']
                    
                    summary_timeframe.loc[summary_timeframe['Date'] == date, 'Profit'] += profit
        
        filtered_summary_timeframe = summary_timeframe[summary_timeframe['Profit'] != 0.0]
        
        return filtered_summary_timeframe
                
    except Exception as e:
        print(f"Error in load_summary: {e}")
        return pd.DataFrame()
    
def stock_plot(all_stocks):
    try:
        for stock in all_stocks:
            ticker = stock['Ticker']
            for info in stock['Informations']:
                timeframe = info['Timeframe']
                plt.figure(figsize=(10, 6))
                plt.plot(timeframe['Date'], timeframe['Profit'], label=f'{ticker} Profit', linewidth=2)
                plt.title(f'Profit Over Time for {ticker}')
                plt.xlabel('Date')
                plt.ylabel('Profit')
                plt.legend()
                plt.grid()
                plt.show()

    except Exception as e:
        print(f"Error in plot_profits: {e}")

def summary_plot(summary):
    try:
        plt.plot(summary['Date'], summary['Profit'], label='Total Profit', linewidth=3, linestyle='--', color='black')
        plt.title('Combined Profit Over Time')
        plt.xlabel('Date')
        plt.ylabel('Profit')
        plt.legend()
        plt.grid()
        plt.show()
    
    except Exception as e:
        print(f"Error in plot_profits: {e}")
        

In [ ]:
if __name__ == "__main__":
    closed_positions = load_file("data/CLOSED POSITIONS.csv")
    open_positions = load_file("data/OPEN POSITIONS.csv")

    df = merge_df(closed_positions, open_positions)
    format_df(df)
    
    unique_tickers = df['Formatted Symbol'].unique()
    
    all_stocks = all_stocks_information(unique_tickers)
    summary = load_summary(all_stocks)

    stock_plot(all_stocks)
    summary_plot(summary)